# Database Query and Heatmap (V2)

Library Initialization

In [20]:
from Modules.database_gen_sqlite import gen_data
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import sqlite3

Database Generation Parameters

In [2]:
wall_width, wall_height = 120, 360
num_studs = 50
base_name = "test1"

Simulation Parameters

In [ ]:
stud_probability = 0.25
base_stud_prob = 0.5

Generate Database

In [5]:
database_path = gen_data(base_name, wall_width, wall_height, num_studs, True)

'/home/anty/projects/heatmap_processing/databases/test1.db'

Extract Data from database

In [8]:
# Connects to the database
con = sqlite3.connect(database_path)
cur = con.cursor()

In [18]:
# Query entire data in table
cur.execute("SELECT * from stud_locations")
data = cur.fetchall()

Save data into array

In [ ]:
stud_array = np.zeros([wall_width, wall_height], dtype=float)

In [ ]:
# Save coordinates to numpy array
for coord in data:
    
   x_coord, y_coord = coord[0], coord[1]
   
   # Check of overlapping studs
   if(stud_array[y_coord, x_coord] > 0 and
      stud_array[y_coord, x_coord] <= 1):

      data[:, x_coord] = (data[y_coord, x_coord] * stud_probability) + data[y_coord,x_coord]
      continue

   stud_array[:, x_coord] = base_stud_prob

Stud Data Refinement